In [6]:
import numpy as np
import yfinance as yf
import scipy.stats as st

### Random state for reproducibility

In [2]:
rs = np.random.RandomState(seed=100)

### Mean, standard deviation and covariance

In [22]:
num_samples = 100
num_stock_returns = 3
X = rs.rand(num_samples, num_stock_returns)  
print(f"Mean:\n", np.mean(X, axis=0))
print(f"Standard deviation:\n", np.std(X, axis=0))

"""
np.cov() assumes that each row is a variable and each column is an observation. Pass rowvar=False to indicate that each column is a variable and each row is an observation.
"""
print(f"Covariance:\n", np.cov(X, rowvar=False, ddof=1))

Mean:
 [0.50020728 0.50375054 0.51967753]
Standard deviation:
 [0.28113481 0.29302848 0.2926356 ]
Covariance:
 [[ 0.07983513 -0.00219274 -0.01044092]
 [-0.00219274  0.08673302  0.00588785]
 [-0.01044092  0.00588785  0.0865006 ]]


### Samples from uniform distribution

In [24]:
rs.rand(2, 2)

array([[0.18899771, 0.51658582],
       [0.80484256, 0.22853715]])

### Samples from standard normal distribution

In [23]:
rs.randn(3, 3)

array([[ 1.02973269e+00,  9.86687347e-03,  9.53420338e-01],
       [-5.55246923e-01,  5.22770324e-01, -1.04070697e+00],
       [ 4.82990601e-04,  1.00350734e+00, -1.24880773e+00]])

### Samples from normal distribution 

In [25]:
mu = 1
sigma = 2
sigma + rs.randn(2, 2) + mu

array([[3.56032623, 3.00476358],
       [3.18166179, 4.42104832]])

### Generating samples from normal distribution using Cholesky decomposition

Affine transformation of a normal random variable:

$$
X \sim N(\mu, \Sigma)
$$
$$
Y = AX + b
$$
$$
Y \sim N(A\mu + b, A\Sigma A^T)
$$

To generate samples from a multivariate normal distribution, we can use the following transformation:

1. Generate $X \sim N(0, I)$
2. Set $A = L$, where $L$ is the Cholesky decomposition of $\Sigma = LL^T$ and b = $\mu$
3. Compute Y = LX + $\mu$ where $Y \sim N(\mu, LIL^T)$ = $N(\mu, \Sigma)$

In [27]:
num_samples = 100
num_stock_returns = 3

X = rs.rand(num_samples, num_stock_returns)  

Sigma = np.cov(X, rowvar=False, ddof=1)
mu = np.mean(X, axis=0)

L = np.linalg.cholesky(Sigma)
Y = L@X.T + mu.reshape(-1, 1)
Y

array([[0.74989373, 0.54527613, 0.73785879, 0.6087066 , 0.54399396,
        0.63401107, 0.6437658 , 0.79034747, 0.6786374 , 0.59824243,
        0.75071186, 0.69060341, 0.6441246 , 0.76240495, 0.62972174,
        0.75084123, 0.77885065, 0.65541737, 0.67764829, 0.73825494,
        0.7316964 , 0.55201835, 0.64669317, 0.59259623, 0.55139142,
        0.67154034, 0.70997029, 0.62379383, 0.7195956 , 0.77451596,
        0.66598697, 0.75213853, 0.73936249, 0.68145896, 0.74485747,
        0.62875531, 0.74681263, 0.69769373, 0.79019423, 0.58335763,
        0.63838649, 0.59529219, 0.57437958, 0.59912234, 0.68810658,
        0.74551029, 0.63036733, 0.70182207, 0.78882885, 0.58339923,
        0.58848954, 0.72930133, 0.6718751 , 0.76943038, 0.55922146,
        0.71297024, 0.70755356, 0.69461204, 0.67328498, 0.58643421,
        0.69513451, 0.62465357, 0.59775809, 0.54140626, 0.5991382 ,
        0.76991325, 0.57126378, 0.76338053, 0.70663008, 0.58927639,
        0.6750378 , 0.53784096, 0.54287794, 0.53

### Fitting distribution to data

**Steps:**
1) Fit a distribution to the data i.e params = st.norm.fit(data)
2) K-S test to compare data with the fitted distribution i.e st.kstest(data, 'norm', params). 
3) The K-S test returns a p-value. The p-value is the probability that the data is consistent with the distribution. The lower the p-value, the less consistent the data is with the distribution.

In [3]:
assets = ['JCI', 'TGT', 'CMCSA', 'CPB', 'MO', 'APA', 'AAPL', 'MSFT', 'NVDA']
assets.sort()

start = '2019-12-30'
end = '2022-12-31'

prices = yf.download(assets, start=start, end=end)
# Here we select all rows and columns that has 'Adj Close' in first level of columns
prices = prices.loc[:, ('Adj Close', slice(None))]
# Here we replace the names of the columns
prices.columns = assets

prices.head()

C:\Users\josep\PycharmProjects\portfolio-convexopt\venv\lib\site-packages\yfinance\utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
C:\Users\josep\PycharmProjects\portfolio-convexopt\venv\lib\site-packages\yfinance\utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
C:\Users\josep\PycharmProjects\portfolio-convexopt\venv\lib\site-packages\yfinance\utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
C:\Users\josep\PycharmProjects\portfolio-convexopt\venv\lib\site-packages\yfinance\utils.py:775: 

,AAPL,APA,CMCSA,CPB,JCI,MO,MSFT,NVDA,TGT
Date,,,,,,,,,
2019-12-30,70.911552,23.763147,40.809372,43.151562,36.968712,35.978863,151.579407,57.851223,117.348251
2019-12-31,71.429665,23.847015,40.619667,43.300510,37.023273,35.993282,151.685181,58.593277,116.729149
2020-01-02,73.059418,23.632681,40.980980,42.485680,37.650799,35.517311,154.493820,59.741241,114.780769
2020-01-03,72.349136,23.940205,40.655815,42.433105,36.977802,35.791359,152.570099,58.785027,113.588081
2020-01-06,72.925629,23.893608,40.348698,42.503189,37.450718,35.935596,152.964462,59.031544,112.513748


In [4]:
R = np.log(prices.shift(-1) / prices)
R = R.dropna()
R.head()

,AAPL,APA,CMCSA,CPB,JCI,MO,MSFT,NVDA,TGT
Date,,,,,,,,,
2019-12-30,0.007280,0.003523,-0.004659,0.003446,0.001475,0.000401,0.000698,0.012745,-0.005290
2019-12-31,0.022560,-0.009029,0.008856,-0.018997,0.016807,-0.013312,0.018347,0.019403,-0.016832
2020-01-02,-0.009770,0.012929,-0.007966,-0.001238,-0.018036,0.007686,-0.012530,-0.016135,-0.010445
2020-01-03,0.007937,-0.001948,-0.007583,0.001650,0.012708,0.004022,0.002581,0.004185,-0.009503
2020-01-06,-0.004714,0.237394,0.010516,0.000623,-0.010497,-0.005231,-0.009159,0.012034,0.001779


In [8]:
def find_best_distribution(data):
    
    dist_names = [ 'norm', 'johnsonsu', 'laplace', 't' ]
    
    dist_params = {}
    results = []
    for dist_name in dist_names:
        dist = getattr(st, dist_name)
        params = dist.fit(data)
        dist_params[dist_name] = params
        res = st.kstest(data, dist_name, params)
        results.append((dist_name, params, res.pvalue))
    
    return max(results, key = lambda x: x[1]) 

for asset in assets:
    print(find_best_distribution(R[asset]))

('t', (4.1644310634749555, 0.001094532163255329, 0.016909377199080713), 0.940393778430694)
('t', (3.085343290419967, 0.0013735996865943396, 0.03175729959582375), 0.9141535581281418)
('t', (3.1505307021504283, 0.0006490346677838425, 0.012793186461814245), 0.9086001431552064)
('t', (3.2531143136473926, 0.0008210959055102358, 0.010650996341175643), 0.6883835648065829)
('t', (3.8395184936026263, 0.0014275331225233295, 0.014976443101616214), 0.9527348545768006)
('t', (2.723905283784589, 0.001406931322080953, 0.010487779870520617), 0.7501216955172719)
('t', (3.6082128636732005, 0.0009863039703548171, 0.014914331953821131), 0.8845275018841077)
('t', (5.936703019567064, 0.0016198530980268285, 0.028760138775080463), 0.790758518501461)
('t', (2.7472339919796402, 0.0007879531232913586, 0.013006322808994499), 0.9718481656173273)
